# Инициализация

In [1]:
import numpy as np
import pandas as pd
import os

In [2]:
import sys

from google.colab import drive
drive.mount('/content/drive')

ROOT = "/content/drive/MyDrive/projects-ds/birds/"
sys.path.append(os.path.join(ROOT, 'code'))

Mounted at /content/drive


In [3]:
from bird_logger import Logger, LogOperation
from bird_progress import ProgressBox
from bird_files import Bird

# Код удаления записей с некорректными значениями

In [34]:
# Расстояние между двумя точками.
# Расстояние приблизительное, т.к. считается по сфере. 
# Но для обнаружения выбросов такого метода вычисления расстояния достаточно,
# а работает это гораздо быстрее, чем расчёт корректных расстояний через shapely/geopandas.
def coord2distance(lat1,lon1,lat2,lon2):
  R = 6371
  dLat = np.radians(lat2-lat1)
  dLon = np.radians(lon2-lon1)
  a = np.sin(dLat/2) * np.sin(dLat/2) + np.cos(np.radians(lat1)) * np.cos(np.radians(lat2)) * np.sin(dLon/2) * np.sin(dLon/2)
  c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
  d = R * c
  return d

# Скорость между текущей точкой и точкой с заданным сдвигом (предыдущей или следующей).
# Вычисляется для всего датафрейма. Размерность - км/ч.
def step_speed(bird, shift=1):
    shift_lon = bird.df.location_long.shift(shift)
    shift_lat = bird.df.location_lat.shift(shift)
    dist = coord2distance(bird.df.location_lat, bird.df.location_long, shift_lat, shift_lon)
    time = abs(bird.df.timestamp.diff(shift).dt.total_seconds()/3600)
    speed = dist / time
    return speed

# Все операции очистки некорректных записей
def clean_bird(bird):
    # удаляем записи, где долгота и/или широта равна NaN
    with LogOperation(LOG, bird, "drop_nan_coords"):
        nan_coord = bird.df.location_long.isnull() | bird.df.location_lat.isnull()
        bird.df.drop(bird.df[nan_coord].index, inplace=True)

    # удаляем записи, где долгота или широта околонулевые
    with LogOperation(LOG, bird, "drop_zero_coords"):
        zero_coord = (bird.df.location_lat.abs()<0.001) & (bird.df.location_long.abs()<0.001)
        bird.df.drop(bird.df[zero_coord].index, inplace=True)

    # удаляем некорректные значения широты > 90
    with LogOperation(LOG, bird, "drop_latitudes_above90"):
        invalid_lat = bird.df.location_lat.abs() > 90
        bird.df.drop(bird.df[invalid_lat].index, inplace=True)

    # удаляем записи, где год больше 2021
    with LogOperation(LOG, bird, "drop_years_above2021"):
        invalid_year = bird.df.timestamp.dt.year>2021
        bird.df.drop(bird.df[invalid_year].index, inplace=True)

    # Удаляем выбросы на траекториях (сбои GPS).
    # Выброс определяем как точку, скорости которой в промежутке 
    # от предыдущей точки и до следующей точки превышают порог.
    SPEED_THRESHOLD = 300
    with LogOperation(LOG, bird, "drop_track_outliers"):
        prev_speed = step_speed(bird, shift=1)
        next_speed = step_speed(bird, shift=-1)
        jumps = (prev_speed>SPEED_THRESHOLD) & (next_speed>SPEED_THRESHOLD)
        bird.df.drop(bird.df[jumps].index, inplace=True)

## Цикл очистки некорректных записей **01_format -> 02_clean**

In [37]:
%%time

SRC_DIR = os.path.join(ROOT, "results/01_format")
SAVE_DIR = os.path.join(ROOT, "results/02_clean")
LOG_DIR = os.path.join(ROOT, "logs")

bird_files = sorted(os.listdir(SRC_DIR))

disp = ProgressBox(stages={'clean':'удаляем плохие записи'}, total_files=len(bird_files))

!rm -r -f $SAVE_DIR/*.*

LOG = Logger(LOG_DIR, prefix="clean_")

for i, file in enumerate(bird_files):
    disp.new_file(file, count=i+1)
    bird = Bird(os.path.join(SRC_DIR, file))
    
    disp.new_stage(stage="clean")
    clean_bird(bird)

    disp.new_stage(stage="save")
    bird.save(SAVE_DIR, "c_")

    disp.new_stage(stage="ok")
    
LOG.flush()

CPU times: user 55.1 s, sys: 9.79 s, total: 1min 4s
Wall time: 1min 6s
